In [3]:
import sys 
sys.path.append('../utils')
from amg_utils import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [ ]:
# 1. pull vanilla data. all GN from tonic combine perf
# -- do need to get first budget. join by adset_id and then just get the first budget, budget_type 
# 2. python functions for campaign actions rules -- this should also generate status per day 
# 3. keywords to vectors 
# 4. add the fb hourly data

In [39]:
# sql 

fb_tonic_daily_perf_query = """   
   
SELECT    
   a.eventdate,   
   a.buyer_initials,
   a.buyer_name,
   a.account_currency,
   a.account_id,
   a.account_name,
   a.ad_id, 
   a.ad_name,
   a.adset_id,
   a.adset_name,
   a.campaign_id,
   a.campaign_name,
 
   a.job_type,
   d.first_budget,
   COALESCE(a.fb_clicks_all, 0) AS fb_clicks_all,
   COALESCE(a.fb_impressions, 0) AS fb_impressions,
   COALESCE(a.fb_leads, 0) AS fb_leads,
   COALESCE(a.fb_link_click, 0) AS fb_link_click,
   a.fb_spend,
   a.rev_clicks,
   a.gross_revenue,
   a.net_revenue,
   a.fb_clicks_all / COALESCE(NULLIF(a.fb_impressions, 0), 1) AS buy_side_ctr,
   a.net_revenue / COALESCE(NULLIF(a.fb_spend, 0), 1) AS roas,
   a.net_revenue / COALESCE(NULLIF(a.fb_clicks_all, 0), 1) AS rpc,
   (a.fb_spend / COALESCE(NULLIF(a.fb_impressions, 0), 1))*1000 AS buy_side_cpm,
   a.fb_spend / COALESCE(NULLIF(a.fb_clicks_all, 0), 1) AS cpc,
   a.net_revenue / COALESCE(NULLIF(a.rev_clicks, 0), 1) AS rpp,
   a.fb_spend / COALESCE(NULLIF(a.rev_clicks, 0), 1) AS cpp,
   a.net_revenue - a.fb_spend AS contrib,    

   b.budget_type,   
   b.status, 

   c.devices,
   c.countries,
   c.network,
   targeting_json:age_max::string AS age_max,
   targeting_json:age_min::string AS age_min,
   targeting_json:facebook_positions::string AS facebook_positions,
   targeting_json:locales::string AS locales  

FROM PRODUCTION.BD_S2CINTERNAL.v_cm_fb_tonic_daily_combine a 

LEFT JOIN SEM_TOOLS.cm_fb_campaign_management.ad_set_latest b 
   ON a.adset_id = b.id  
   AND a.campaign_id = b.campaign_id 
   AND a.adset_name = b.name

LEFT JOIN PRODUCTION.BD_S2CINTERNAL.V_FACEBOOK_ADSET_METADATA c 
   ON a.adset_id = c.id 
   AND a.eventdate = c.event_date

LEFT JOIN (
   SELECT id, MIN(updated) AS first_updated, MIN(budget) AS first_budget
   FROM sem_tools.CM_FB_CAMPAIGN_MANAGEMENT.ad_set
   GROUP BY id
) AS d ON a.adset_id = d.id

WHERE a.buyer_initials = 'GN'   
and job_type = 'final'

ORDER BY  a.ad_id ASC, a.eventdate ASC  
   """
fb_tonic_daily_perf = eq(fb_tonic_daily_perf_query)

In [40]:
# fb_tonic_daily_perf.to_csv('fb_tonic_daily_perf.csv', index=False)

In [257]:
fb_tonic_daily_perf = pd.read_csv("fb_tonic_daily_perf.csv", parse_dates=['eventdate'])

In [258]:
# Crate campaign duration in days variable
fb_tonic_daily_perf.insert(2, 'campaign_duration', fb_tonic_daily_perf.groupby('campaign_id')['eventdate'].transform(lambda x: (x.max() - x.min()).days))

# Create campaign end date variable
fb_tonic_daily_perf.insert(2, 'campaign_end_date', fb_tonic_daily_perf.groupby('campaign_id')['eventdate'].transform("max"))

fb_tonic_daily_perf.insert(2, 'fb_created', fb_tonic_daily_perf.groupby('campaign_id')['eventdate'].transform("min"))


# Filter out campaigns with less than {min_campaign_duration} days duration
min_campaign_duration = 4 
series = fb_tonic_daily_perf[fb_tonic_daily_perf['campaign_duration'] > 4].copy()
series.ad_id = series.ad_id.astype('str')

static_cols = [      
"network",
"devices"]   

def one_hot_encode(df, cols):
    encoded = pd.get_dummies(df[cols])
    df = df.drop(columns=cols, axis=1)
    df = pd.concat([df, encoded], axis=1)
    return df

series =  one_hot_encode(series, static_cols)

In [51]:
## stats 

dataset = "gn_fb_tonic"
print(f"Dataset: {dataset}")
# Total number of campaigns
print(f"Total number of campaigns: {series.ad_id.nunique()}")
# Ads starting from date ... 
print(f"Min eventdate: {series.eventdate.min().date()}") 
print(f"Max eventdate: {series.eventdate.max().date()}")
days_between = (series.eventdate.max().date() - series.eventdate.min().date()).days
print(f"Days between min and max eventdate: {days_between}")
print()
# Duration of campaigns 
campaign_duration_count = series.groupby(["ad_id", "campaign_duration"]).size().value_counts().sort_index().reset_index(name='count').rename(columns={'index':'campaign_duration (days)'})
print(f"Duration of campaigns (filtered to only campaigns of {min_campaign_duration} or more days): \n{campaign_duration_count}")
print()
# mean campaign duration 
print(f"Mean campaign duration: {series.campaign_duration.mean().round(1)} days")

series['fb_created_date'] = pd.to_datetime(series['fb_created']).dt.date
print()

# number of unique ad_ids per date 
campaigns_created_per_date = series.groupby('fb_created_date').ad_id.nunique().reset_index(name='count').sort_values('fb_created_date', ascending=True)
print(f"Number of campaigns created per date: \n {campaigns_created_per_date}")
positive_contrib_ads = series.groupby('ad_id')['contrib'].sum().reset_index()
print()
positive_contrib_ads = positive_contrib_ads[positive_contrib_ads['contrib'] > 0]['ad_id']
print(f"Number of campaigns with positive contribution: \n {positive_contrib_ads.nunique()}")

Dataset: gn_fb_tonic
Total number of campaigns: 186
Min eventdate: 2024-01-17
Max eventdate: 2024-02-23
Days between min and max eventdate: 37

Duration of campaigns (filtered to only campaigns of 4 or more days): 
   campaign_duration (days)  count
0                         5      3
1                         6     18
2                         7     44
3                         8    106
4                         9     10
5                        10      2
6                        15      1
7                        16      1
8                        21      1

Mean campaign duration: 7.2 days

Number of campaigns created per date: 
    fb_created_date  count
0       2024-01-17      2
1       2024-01-18     10
2       2024-01-19     10
3       2024-01-23      3
4       2024-01-24     27
5       2024-01-25     24
6       2024-01-26     24
7       2024-01-27      7
8       2024-01-30     37
9       2024-01-31      2
10      2024-02-16     38
11      2024-02-17      2

Number of campaigns w

current rules 

| Rule                | Logic                                                                                      |
|---------------------|-------------------------------------------------------------------------------------------|
| Pause rule          | if age >= 7 and ROAS < 1 for last 3 days (excluding today), then permanent pause the adset |
| Budget increase rule| if ROAS > 1 for last 2 days (excluding today) and daily budget < $100, then double daily budget |
| Budget decrease rule| if ROAS < 1 for last 2 days (excluding today) and daily budget > $10, then half daily budget |
| Extend rule         | If not paused, then extend lifetime by one day                                             |


In [1]:
# series_org = series.copy()
series = series_org.copy()

NameError: name 'series_org' is not defined

In [379]:
def update_pause_and_extend_campaign_conditions(series):
    campaign_duration_condition = (series['eventdate'] - series['fb_created']).dt.days >= 7
    roas_condition_pause = series.groupby('ad_id')['roas'].transform(lambda x: (x.shift(1) < 1) & (x.shift(2) < 1) & (x.shift(3) < 1))    
    series['pause_campaign'] = (campaign_duration_condition & roas_condition_pause).fillna(False).astype(bool)    
    series['extend_campaign'] = ~series['pause_campaign']
    
    return series

In [380]:
def generate_autobid_sequence(series):
    series['budget'] = series.groupby('ad_id')['first_budget'].transform('first').astype(float)
    return (series
            .pipe(update_pause_and_extend_campaign_conditions)
            .pipe(update_budgets_based_on_conditions))

series = generate_autobid_sequence(series)

In [356]:
peek(series.query("adset_id == 120205035015970410"))

,,,,,,,,,,
0,eventdate,datetime64[ns],2024-02-16 00:00:00,2024-02-17 00:00:00,2024-02-18 00:00:00,2024-02-19 00:00:00,2024-02-20 00:00:00,2024-02-21 00:00:00,2024-02-22 00:00:00,2024-02-23 00:00:00
1,buyer_initials,object,GN,GN,GN,GN,GN,GN,GN,GN
2,fb_created,datetime64[ns],2024-02-16 00:00:00,2024-02-16 00:00:00,2024-02-16 00:00:00,2024-02-16 00:00:00,2024-02-16 00:00:00,2024-02-16 00:00:00,2024-02-16 00:00:00,2024-02-16 00:00:00
3,campaign_end_date,datetime64[ns],2024-02-23 00:00:00,2024-02-23 00:00:00,2024-02-23 00:00:00,2024-02-23 00:00:00,2024-02-23 00:00:00,2024-02-23 00:00:00,2024-02-23 00:00:00,2024-02-23 00:00:00
4,campaign_duration,int64,7,7,7,7,7,7,7,7
5,buyer_name,object,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets
6,account_currency,object,USD,USD,USD,USD,USD,USD,USD,USD
7,account_id,int64,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781
8,account_name,object,fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles)
9,ad_id,object,120205035019250410,120205035019250410,120205035019250410,120205035019250410,120205035019250410,120205035019250410,120205035019250410,120205035019250410


In [376]:
peek(series.query("adset_id == 120205035015970410")[['eventdate', 'roas', 'first_budget', 'budget', 'budget_increase', 'budget_decrease', 'pause_campaign', 'extend_campaign']])

,,,,,,,,,,
0,eventdate,datetime64[ns],2024-02-16 00:00:00,2024-02-17 00:00:00,2024-02-18 00:00:00,2024-02-19 00:00:00,2024-02-20 00:00:00,2024-02-21 00:00:00,2024-02-22 00:00:00,2024-02-23 00:00:00
1,roas,float64,0.227036,0.38412,0.478805,0.2871,0.150711,0.409091,0.506612,0.686957
2,first_budget,int64,1000,1000,1000,1000,1000,1000,1000,1000
3,budget,float64,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
4,budget_increase,bool,False,False,False,False,False,False,False,False
5,budget_decrease,bool,False,False,False,False,False,False,False,False
6,pause_campaign,bool,False,False,False,False,False,False,False,True
7,extend_campaign,bool,True,True,True,True,True,True,True,False


In [353]:
check_table(name="sem_tools.CM_FB_CAMPAIGN_MANAGEMENT.cm_fb_tonic_recommendation ")

,,,,,,,
0,eventdate,object,2024-02-16,2024-02-16,2024-02-16,2024-02-16,2024-02-16
1,account_id,object,None,None,None,None,None
2,campaign_id,int64,120205035015640410,120205035010890410,120205037230460410,120205037230360410,120205037238600410
3,campaign_name,object,None,None,None,None,None
4,adset_id,object,120205035015970410,120205035011420410,120205037230780410,120205037230510410,120205037239940410
5,rule,object,"{""name"": ""daily_fb_tonic_budget_increase_rule""...","{""name"": ""daily_fb_tonic_budget_increase_rule""...","{""name"": ""daily_fb_tonic_budget_increase_rule""...","{""name"": ""daily_fb_tonic_budget_increase_rule""...","{""name"": ""daily_fb_tonic_budget_increase_rule""..."
6,recommendation,object,"{""name"": ""NO_ACTION""}","{""name"": ""NO_ACTION""}","{""name"": ""NO_ACTION""}","{""name"": ""NO_ACTION""}","{""name"": ""NO_ACTION""}"
7,event,object,None,None,None,None,None
8,request_id,object,test,test,test,test,test


In [9]:
t = eq("select * from sem_tools.CM_FB_CAMPAIGN_MANAGEMENT.cm_fb_tonic_recommendation where adset_id = 120205035015970410")


In [11]:
peek(t)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,eventdate,object,2024-02-16,2024-02-16,2024-02-16,2024-02-17,2024-02-17,2024-02-17,2024-02-17,2024-02-17,2024-02-17,2024-02-17,2024-02-17,2024-02-17,2024-02-17,2024-02-17,2024-02-17,2024-02-18,2024-02-18,2024-02-18,2024-02-19,2024-02-19,2024-02-19,2024-02-20,2024-02-20,2024-02-20,2024-02-21,2024-02-21,2024-02-21,2024-02-22,2024-02-22,2024-02-22,2024-02-23,2024-02-23,2024-02-23
1,account_id,object,None,None,None,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781
2,campaign_id,int64,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410,120205035015640410
3,campaign_name,object,None,None,None,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...,AdcopyTest|20240215|Variant3|Juice for Kidney ...
4,adset_id,object,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410,120205035015970410
5,rule,object,"{""name"": ""daily_fb_tonic_budget_increase_rule""...","{""name"": ""daily_fb_tonic_pause_rule"", ""cadence...","{""name"": ""daily_fb_tonic_budget_decrease_rule""...","{""name"": ""daily_fb_t

In [363]:
t.iloc[2].recommendation

'{"name": "DECREASE_BUDGET"}'

In [364]:
t.iloc[2].event

'{"adSetId": "120205035015970410", "budget": {"budget": 1000, "budgetType": "DAILY"}, "endTime": "2024-02-23T00:00:00-0800"}'

In [382]:
series.to_csv('series_2.csv', index=False)

In [ ]:
peek(series.query("adset_id ==  120205035015970410 ")[['eventdate', 'roas', 'first_budget', 'budget', 'budget_increase', 'budget_decrease', 'pause_campaign', 'extend_campaign']])

,,,,,,,,,,
0,eventdate,datetime64[ns],2024-02-16 00:00:00,2024-02-17 00:00:00,2024-02-18 00:00:00,2024-02-19 00:00:00,2024-02-20 00:00:00,2024-02-21 00:00:00,2024-02-22 00:00:00,2024-02-23 00:00:00
1,roas,float64,0.227036,0.38412,0.478805,0.2871,0.150711,0.409091,0.506612,0.686957
2,first_budget,int64,1000,1000,1000,1000,1000,1000,1000,1000
3,budget,float64,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
4,budget_increase,bool,False,False,False,False,False,False,False,False
5,budget_decrease,bool,False,False,False,False,False,False,False,False
6,pause_campaign,bool,False,False,False,False,False,False,False,True
7,extend_campaign,bool,True,True,True,True,True,True,True,False


In [ ]:
peek(series.query("adset_id ==  120203732421750410 ")[['eventdate', 'roas', 'first_budget', 'budget', 'budget_increase', 'budget_decrease', 'pause_campaign', 'extend_campaign']])

,,,,,,,,,,,,,,,,,,,,,,,
0,eventdate,datetime64[ns],2024-01-18 00:00:00,2024-01-19 00:00:00,2024-01-20 00:00:00,2024-01-21 00:00:00,2024-01-22 00:00:00,2024-01-23 00:00:00,2024-01-24 00:00:00,2024-01-25 00:00:00,2024-01-26 00:00:00,2024-01-27 00:00:00,2024-01-28 00:00:00,2024-01-29 00:00:00,2024-01-30 00:00:00,2024-01-31 00:00:00,2024-02-01 00:00:00,2024-02-02 00:00:00,2024-02-03 00:00:00,2024-02-04 00:00:00,2024-02-05 00:00:00,2024-02-06 00:00:00,2024-02-07 00:00:00
1,roas,float64,2.556757,1.718391,0.80226,2.666667,0.908127,1.241758,1.350515,1.903304,1.468085,1.169591,1.142116,0.898527,1.024718,1.016468,1.077771,1.046727,1.064642,0.941179,0.790639,0.769313,0.595
2,first_budget,int64,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188
3,budget,float64,188.0,188.0,376.0,376.0,376.0,376.0,376.0,752.0,1504.0,3008.0,6016.0,12032.0,12032.0,12032.0,12032.0,12032.0,12032.0,12032.0,12032.0,6016.0,3008.0
4,budget_increase,bool,False,False,True,False,False,False,False,True,True,True,True,True,False,False,False,False,False,False,False,False,False
5,budget_decrease,bool,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True
6,pause_campaign,bool,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
7,extend_campaign,bool,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False


In [ ]:
peek(eq("select * from sem_tools.CM_FB_CAMPAIGN_MANAGEMENT.ad_set where id = 120203732421750410"))

\
0                 id                int64                120203732421750410   
1               name               object   batch_01|TEST_5|20240117|Adset1   
2        campaign_id                int64                120203732421660410   
3             status               object                            ACTIVE   
4             budget                int64                               188   
5        budget_type               object                             DAILY   
6    review_feedback               object                              None   
7         account_id                int64                  7150655894950781   
8         fb_created  datetime64[ns, UTC]         2024-01-18 01:15:01+00:00   
9         fb_updated  datetime64[ns, UTC]         2024-01-18 01:15:49+00:00   
10           updated  datetime64[ns, UTC]  2024-01-19 20:40:35.266086+00:00   
11  budget_remaining                int64                                86   
12        start_time  datetime64[ns, UTC]         2024-01-18 01:15:01+00:00   
13          end_time  datetime64[ns, UTC]         2024-01-25 01:12:27+00:00   
14        bid_amount               object                              None   

                                                                        \
0                 120203732421750410                120203732421750410   
1    batch_01|TEST_5|20240117|Adset1   batch_01|TEST_5|20240117|Adset1   
2                 120203732421660410                120203732421660410   
3                             ACTIVE                            ACTIVE   
4                                188                               188   
5                              DAILY                             DAILY   
6                               None                              None   
7                   7150655894950781                  7150655894950781   
8          2024-01-18 01:15:01+00:00         2024-01-18 01:15:01+00:00   
9          2024-01-18 01:15:49+00:00         2024-01-18 01:15:49+00:00   
10  2024-01-19 20:50:18.520858+00:00  2024-01-19 21:00:40.959559+00:00   
11                                86                                85   
12         2024-01-18 01:15:01+00:00         2024-01-18 01:15:01+00:00   
13         2024-01-25 01:12:27+00:00         2024-01-25 01:12:27+00:00   
14                              None                              None   

                                                                        \
0                 120203732421750410                120203732421750410   
1    batch_01|TEST_5|20240117|Adset1   batch_01|TEST_5|20240117|Adset1   
2                 120203732421660410                120203732421660410   
3                             ACTIVE                            ACTIVE   
4                                188                               188   
5                              DAILY                             DAILY   
6                               None                              None   
7                   7150655894950781                  7150655894950781   
8          2024-01-18 01:15:01+00:00         2024-01-18 01:15:01+00:00   
9          2024-01-18 01:15:49+00:00         2024-01-18 01:15:49+00:00   
10  2024-01-19 21:20:30.754879+00:00  2024-01-19 21:40:21.334859+00:00   
11                                85                                85   
12         2024-01-18 01:15:01+00:00         2024-01-18 01:15:01+00:00   
13         2024-01-25 01:12:27+00:00         2024-01-25 01:12:27+00:00   
14                              None                              None   

                                                                        \
0                 120203732421750410                120203732421750410   
1    batch_01|TEST_5|20240117|Adset1   batch_01|TEST_5|20240117|Adset1   
2                 120203732421660410                120203732421660410   
3                             ACTIVE                            ACTIVE   
4                                188                               188  